# Escaly Activation Funnel — SQL Notebook
This notebook creates a mock SQLite database for Escaly events and runs SQL to compute activation and funnel metrics.
**Activation definition:** First `GenerateReport` within 7 days of `Signup`.

Cohorts:
- A: Baseline onboarding
- B: Interactive onboarding + AI assistant

In [ ]:

import sqlite3, pandas as pd
conn = sqlite3.connect('/mnt/data/escaly_activation.db')

# Activation within 7 days
activation_sql = """
WITH first_signup AS (
  SELECT user_id, cohort, DATETIME(signup_ts) AS signup_ts
  FROM users
),
gen_report AS (
  SELECT e.user_id, MIN(DATETIME(e.ts)) AS first_report_ts
  FROM events e
  WHERE e.event_name = 'GenerateReport'
  GROUP BY e.user_id
),
activated AS (
  SELECT s.user_id, s.cohort,
         CASE 
           WHEN r.first_report_ts IS NOT NULL 
                AND JULIANDAY(r.first_report_ts) - JULIANDAY(s.signup_ts) <= 7
           THEN 1 ELSE 0 END AS is_activated
  FROM first_signup s
  LEFT JOIN gen_report r ON r.user_id = s.user_id
)
SELECT cohort,
       COUNT(*) AS users,
       SUM(is_activated) AS activated_users,
       ROUND(100.0 * SUM(is_activated) / COUNT(*), 1) AS activation_rate_pct
FROM activated
GROUP BY cohort
ORDER BY cohort;
"""
activation_df = pd.read_sql_query(activation_sql, conn)
activation_df

In [ ]:

# Multi-step funnel summary per cohort
funnel_sql = """
WITH steps AS (
  SELECT u.cohort, u.user_id,
         MAX(CASE WHEN e.event_name='Signup' THEN 1 ELSE 0 END) AS step_signup,
         MAX(CASE WHEN e.event_name='CreateUser' THEN 1 ELSE 0 END) AS step_create_user,
         MAX(CASE WHEN e.event_name='SelectScale' THEN 1 ELSE 0 END) AS step_select_scale,
         MAX(CASE WHEN e.event_name='SubmitAssessment' THEN 1 ELSE 0 END) AS step_submit_assessment,
         MAX(CASE WHEN e.event_name='GenerateReport' THEN 1 ELSE 0 END) AS step_generate_report
  FROM users u
  LEFT JOIN events e ON e.user_id = u.user_id
  GROUP BY u.cohort, u.user_id
)
SELECT cohort,
       SUM(step_signup) AS signup,
       SUM(step_create_user) AS created_user,
       SUM(step_select_scale) AS selected_scale,
       SUM(step_submit_assessment) AS submitted_assessment,
       SUM(step_generate_report) AS generated_report
FROM steps
GROUP BY cohort
ORDER BY cohort;
"""
funnel_df = pd.read_sql_query(funnel_sql, conn)
funnel_df

## Notes
- Replace mock data with your production tables once available.
- Keep the activation definition stable for trend analysis.
- Add a `shared_report` event and test its lift on conversion.